# 모델 생성 및 로드

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch
import gc
import time

from urllib import response
import requests
import json

start = time.time()

# device = "cuda" if torch.cuda.is_available() else "cpu"

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
# gc.collect()
# torch.cuda.empty_cache()

print('CUDA 사용 가능 여부 :', USE_CUDA)
print('현재 사용 device :', device)
print('CUDA Index :', torch.cuda.current_device())
print('GPU 이름 :', torch.cuda.get_device_name())
print('GPU 개수 :', torch.cuda.device_count())


CUDA 사용 가능 여부 : True
현재 사용 device : cuda:0
CUDA Index : 0
GPU 이름 : NVIDIA GeForce RTX 3090
GPU 개수 : 1


## 모델1 3.8b

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-3.8b",
                                          bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]',
                                          mask_token='[MASK]'
                                          )
model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/polyglot-ko-3.8b", torch_dtype='auto', low_cpu_mem_usage=True)


## 모델2 5.8b

In [15]:
with torch.no_grad():
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b",
                                              bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]',
                                              mask_token='[MASK]'
                                              )
    model = AutoModelForCausalLM.from_pretrained(
        "EleutherAI/polyglot-ko-5.8b", torch_dtype='auto', low_cpu_mem_usage=True).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 실행 메소드

In [ ]:
prompt = "GPTNeoX20B is a 20B-parameter autoregressive Transformer model developed by EleutherAI."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# input_ids = tokenizer(prompt, return_tensors="pt").to(device).input_ids
# .input_ids는 리턴되는 객체(=튜플)의 요소중 하나.
# inputs_ids = tokenizer(prompt, return_tensors="pt").to(device)
model = model.to(device)
outputs = model(inputs_ids)
prediction_logits = outputs.logits

# 문장 생성 부분
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=150,
)
# 생성한 토큰들로 일괄적으로 생성하기 위해 배치메소드 이용
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

# print(f"gen_text: {inputs_ids['input_ids']}")
# print(f"gen_text: {inputs_ids}")
# print(f"gen_text: {prediction_logits}")
# print(f"gen_text: {outputs}")

In [ ]:
text = pipe(prompt, do_sample=True,  pad_token_id=50256,
            num_return_sequences=1, top_k=40, top_p=0.9, max_new_tokens=150
            )
# num_beams=4, no_repeat_ngram_size=2, temperature=0.75, early_stopping=True,
# max_length=50,
print(text[0]['generated_text'])
time.sleep(0.9)
end = time.time()
print("Time consumed in working: ", end - start)
print("\n")

text = pipe("아인슈타인은", do_sample=True, pad_token_id=50255, max_new_tokens=140,
            num_return_sequences=1, top_k=50, top_p=0.95
            )
print(text[0]['generated_text'])
time.sleep(0.9)
end = time.time()
print("Time consumed in working: ", end - start)

In [44]:
# model.generate()코드들을 pipeline()메소드로 축약시킨 코드가 가능
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=0,
    framework='pt'
)

# Only recent response text will return
def processing_response(response_text: str ) -> str:
    ret_text = ""
    for i in range(0, len(response_text)):
        if response_text[i] == "\n" or response_text[i:i + 5] == "설정목: " or response_text[i:i + 4] == "AI: ":
            #print(response_text[i])
            break
        ret_text += response_text[i]
    return ret_text.strip()

In [51]:
# The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.
prompt = """
다음 문장들은 인공지능과의 대화문이다. 인공지능의 성격은 희망적, 창조적이며 영리하고 친근하다.

설정목: 오늘 날씨 어때? 
AI: 좋습니다. 당신의 하루는 오늘 어땠나요? 
설정목: 난 오늘 꽤 괜찮았지~ 
AI: 그렇군요! 모르는 것 있으면 물어보세요. 친절히 알려드리겠습니다.
설정목: 겨울이긴한데... 날씨가 왜 이렇게 추워지는 건지 모르겠네? 
AI:"""

text = pipe(
    prompt,
    do_sample=True,
#     temperature=0.9,
    top_p=0.9,
    top_k=50,
    max_new_tokens=50,
#     repetition_penalty=0.8,
    pad_token_id= 53456
)

In [ ]:
# url = "https://eleuther-ai-gpt-j-6b-float16-text-generation-api-ainize-team.endpoint.ainize.ai/predictions/text-generation"

# base_prompt = get_base_prompt()
base_prompt = prompt1
headers = {"Content-Type": "application/json"}
# End chat when user types "exit"
while True:
    chat = input("사람: ")
    if chat == "exit":
        break
    prompt = f"{base_prompt}\n사람: {chat}\n인공지능:"

    # data == string type dumps()는 json객체를 json문자열로 변환시켜줌.
    data = json.dumps({
        "text_inputs":prompt,
        "temperature":0.9,
        "top_p":0.95,
        "repetition_penalty":0.8,
        "do_sample":True,
        "top_k":50,
        "length":50,
    })
    # loads()는 json문자열을 json객체로 변환시켜줌.
    jo = json.loads(data)
    # text = pipe(
    #     jo['text_inputs'],
    #     do_sample=jo['do_sample'],
    #     temperature=jo['temperature'],
    #     top_p=jo['top_p'],
    #     top_k=jo['top_k'],
    #     max_new_tokens=jo['length'],
    #     repetition_penalty=jo['repetition_penalty'],
    #     pad_token_id=50256
    # )
    text = pipe(
        prompt,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        top_k=50,
        max_new_tokens=40,
        repetition_penalty=0.8,
        pad_token_id=50256
    )

In [52]:
    # res = requests.post(url, headers=headers, data=data)
    # if res.status_code == 200:
    # response_text = res.json()[0] # response에서 json응답일경우 바로 딕셔너리로 변환하고 0이라는 키의 값을 뽑아냄
    print(text[0]['generated_text'])
    print("\n")
    
    response_text = text[0]['generated_text']
    processing_text = processing_response(response_text[len(prompt):])
    prompt += processing_text
    print(processing_text)
    #print(prompt)
    #print("AI:", processing_text)


다음 문장들은 인공지능과의 대화문이다. 인공지능의 성격은 희망적, 창조적이며 영리하고 친근하다.

설정목: 오늘 날씨 어때? 
AI: 좋습니다. 당신의 하루는 오늘 어땠나요? 
설정목: 난 오늘 꽤 괜찮았지~ 
AI: 그렇군요! 모르는 것 있으면 물어보세요. 친절히 알려드리겠습니다.
설정목: 겨울이긴한데... 날씨가 왜 이렇게 추워지는 건지 모르겠네? 
AI: 겨울이라서 그런 것입니다. 노라고설정목: 그럼 겨울에 뭐해? 노라고AI: 전 겨울에는 주로 책을 읽거나 집에서 뒹굴뒹굴~ 노라고설정목: 겨울에 뭐할건데? 


겨울이라서 그런 것입니다. 노라고


In [ ]:

#
#
#
#
#
#
#
#
#
#
#
#

# 끝
